In [3]:
"""
Template for week 13 project in Data Visualization

Plot various triangulated surfaces of digital elavation model data for Grand Canyon
https://pubs.usgs.gov/ds/121/grand/grand.html
"""

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff


In [4]:
# Resource paths
PLOTS_PATH = "plots/"
DATA_PATH = "data/"
GC_DEM = DATA_PATH + "gc_dem.tiff"

DEMS = {}

# Sub-region of interest defined as numpy slice objects
REGION = np.s_[1500:1800, 1300:1600]

# Custom colorscale/colormap for elevations
ELEV = ("rgb(5,10,172)",
        "rgb(34,46,193)",
        "rgb(63,83,215)",
        "rgb(92,119,236)",
        "rgb(134,155,228)",
        "rgb(190,190,190)",
        "rgb(220,170,132)", 
        "rgb(230,145,90)",
        "rgb(213,100,69)",
        "rgb(195,55,49)",
        "rgb(178,10,28)")


In [5]:
###############################################################################
# Provided code from Project 12

def load_dem(dem_file):
    """
    Input: String dem_file
    
    Output: Numpy array of integer heights
    
    NOTE: The loaded height are in decimeters. Divide by 10 to
    return integer heights in meters
    """
    gc_image = plt.imread(dem_file)
    dem_array = np.array(gc_image) // 10

    return dem_array

In [6]:
def compute_features(dem_array):
    """
    Input: Numpy array dem_array
    
    Output: Numpy array with boundary rows and columns trimmed
    """    
    float_dem = dem_array.astype(float)
    
    # Use numpy-friendly operations for speed
    horiz_error = float_dem[1 : -1, : -2] - 2 * float_dem[1 : -1, 1 : -1] + float_dem[1 : -1, 2 :]
    vert_error = float_dem[: -2, 1 : -1] - 2 * float_dem[1 : -1, 1 : -1] + float_dem[2 :, 1 : -1]
    
    feature_array = np.abs(horiz_error) + np.abs(vert_error)
    return feature_array

In [7]:
def create_dems():
    """ Create some example dems for testing/plotting """
    
    # Small examples
    DEMS["2x2"] = np.array([[1, 2], [3, 4]])
    DEMS["3x5"] = np.array([[1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7]])
    
    # Medium examples
    x_grid, y_grid = np.meshgrid(np.linspace(-1, 1, 9), np.linspace(-1, 1, 9))
    ridge1 = np.minimum(x_grid - y_grid, -x_grid + y_grid)
    ridge2 = np.minimum(x_grid + y_grid, -x_grid - y_grid)
    DEMS["ridge"] = np.maximum(ridge1, ridge2)
    
    x_grid, y_grid = np.meshgrid(np.linspace(-1, 1, 21), np.linspace(-2, 2, 41))
    DEMS["error"] = x_grid ** 2 - y_grid ** 2
    
    # Flip up/down due to difference in coordinate systems between image and 3D plots
    gc_dem = load_dem(GC_DEM)
    DEMS["region"] = np.flipud(gc_dem[REGION])

create_dems()

In [8]:
####################################################################
# Part 1 - Generate topology of quad mesh for given grid shape

def make_quads(grid_shape):
    """
    Input: Tuple grid_shape consisting size of y and x dimensions of grid respectively
    
    Output: Tuple consisting of 2D array of 2D vertex positions
    and list of quads represented as tuples of vertex indices
    
    NOTE: The ordering of the returned vertex indices must be consistent 
    with the order returned by the ravel() method
    """    
    position = []
    quad = []
    
    # get the position x,y
    for idx_j in range(grid_shape[0]):
        for idx_i in range(grid_shape[1]):
            position.append([idx_i, idx_j])
    
    #initialized the grid
    grid = np.arange(grid_shape[0] * grid_shape[1]).reshape(grid_shape)

    # get the quad using indexing 
    quad1 = grid[:-1, :-1]
    quad2 = grid[:-1, 1::]
    quad3 = grid[1::, 1::]
    quad4 = grid[1::, :-1]
    
    #get the number for each quad
    for idx_i in range(quad1.shape[0]):
        for idx_j in range(quad1.shape[1]):
            quad.append((quad1[idx_i][idx_j], quad2[idx_i][idx_j], 
                         quad3[idx_i][idx_j], quad4[idx_i][idx_j]))
    
    return (np.array(position), np.array(quad))

In [9]:
def test_make_quads():
    """ Test make_quads """
    
    print(make_quads((1, 1)))
    print(make_quads((2, 2)))
    print(make_quads((2, 3)))
    print(make_quads((3, 2)))

test_make_quads()

(array([[0, 0]]), array([], dtype=float64))
(array([[0, 0],
       [1, 0],
       [0, 1],
       [1, 1]]), array([[0, 1, 3, 2]]))
(array([[0, 0],
       [1, 0],
       [2, 0],
       [0, 1],
       [1, 1],
       [2, 1]]), array([[0, 1, 4, 3],
       [1, 2, 5, 4]]))
(array([[0, 0],
       [1, 0],
       [0, 1],
       [1, 1],
       [0, 2],
       [1, 2]]), array([[0, 1, 3, 2],
       [2, 3, 5, 4]]))


Correct output
~~~~
(array([[0, 0]]), [])
(array([[0, 0],
       [1, 0],
       [0, 1],
       [1, 1]]), [(0, 1, 3, 2)])
(array([[0, 0],
       [1, 0],
       [2, 0],
       [0, 1],
       [1, 1],
       [2, 1]]), [(0, 1, 4, 3), (1, 2, 5, 4)])
(array([[0, 0],
       [1, 0],
       [0, 1],
       [1, 1],
       [0, 2],
       [1, 2]]), [(0, 1, 3, 2), (2, 3, 5, 4)])
~~~~

In [60]:
##################################################################
# Part 2 - Generate triangulations of elevation maps

def make_trimesh_fixed(z_grid, diagonal="ul_lr"):
    """
    Input: 2D numpy array z_grid of elevation values, optional string diagonal
    that specifies direction of quad diagonal - "ul_lr" or "ll_ur"
    
    Output: Tuple consisting of 2D numpy array of 3D vertex positions
    and a list of triangles represented as tuples of vertex indices
    """
    
    positions, quads = make_quads(z_grid.shape)
    
    tri = []
    verts = []
    
    idx = 0 
    for idx_i in range(z_grid.shape[0]):
        for idx_j in range(z_grid.shape[1]):
            verts.append([positions[idx][0], positions[idx][1], z_grid[idx_i][idx_j]])
            idx += 1
            
    if diagonal == "ul_lr":
        for quad in quads:
            tri.append((quad[0], quad[1], quad[2]))
            tri.append((quad[2], quad[3], quad[0]))
    else:
        for quad in quads:
            tri.append((quad[3], quad[0], quad[1]))
            tri.append((quad[1], quad[2], quad[3]))
    
    return (np.array(verts), np.array(tri))

In [11]:
def test_make_trimesh_fixed():
    """ Test make_trimesh_fixed() """
    
    print(make_trimesh_fixed(DEMS["2x2"]))
    print(make_trimesh_fixed(DEMS["2x2"], diagonal="ll_ur"))
    print(make_trimesh_fixed(DEMS["3x5"], diagonal="ul_lr"))
    print(make_trimesh_fixed(DEMS["3x5"], diagonal="ll_ur"))

    make_trimesh_fixed(DEMS["ridge"])
    make_trimesh_fixed(DEMS["ridge"], diagonal="ll_ur")
    make_trimesh_fixed(DEMS["error"])      
    make_trimesh_fixed(DEMS["region"])
    

test_make_trimesh_fixed()

(array([[0, 0, 1],
       [1, 0, 2],
       [0, 1, 3],
       [1, 1, 4]]), array([[0, 1, 3],
       [3, 2, 0]]))
(array([[0, 0, 1],
       [1, 0, 2],
       [0, 1, 3],
       [1, 1, 4]]), array([[2, 0, 1],
       [1, 3, 2]]))
(array([[0, 0, 1],
       [1, 0, 2],
       [2, 0, 3],
       [0, 1, 2],
       [1, 1, 3],
       [2, 1, 4],
       [0, 2, 3],
       [1, 2, 4],
       [2, 2, 5],
       [0, 3, 4],
       [1, 3, 5],
       [2, 3, 6],
       [0, 4, 5],
       [1, 4, 6],
       [2, 4, 7]]), array([[ 0,  1,  4],
       [ 4,  3,  0],
       [ 1,  2,  5],
       [ 5,  4,  1],
       [ 3,  4,  7],
       [ 7,  6,  3],
       [ 4,  5,  8],
       [ 8,  7,  4],
       [ 6,  7, 10],
       [10,  9,  6],
       [ 7,  8, 11],
       [11, 10,  7],
       [ 9, 10, 13],
       [13, 12,  9],
       [10, 11, 14],
       [14, 13, 10]]))
(array([[0, 0, 1],
       [1, 0, 2],
       [2, 0, 3],
       [0, 1, 2],
       [1, 1, 3],
       [2, 1, 4],
       [0, 2, 3],
       [1, 2, 4],
       [2, 2, 5],


Correct output
~~~~
(array([[0, 0, 1],
       [1, 0, 2],
       [0, 1, 3],
       [1, 1, 4]]), [(0, 1, 3), (3, 2, 0)])
(array([[0, 0, 1],
       [1, 0, 2],
       [0, 1, 3],
       [1, 1, 4]]), [(2, 0, 1), (1, 3, 2)])
(array([[0, 0, 1],
       [1, 0, 2],
       [2, 0, 3],
       [0, 1, 2],
       [1, 1, 3],
       [2, 1, 4],
       [0, 2, 3],
       [1, 2, 4],
       [2, 2, 5],
       [0, 3, 4],
       [1, 3, 5],
       [2, 3, 6],
       [0, 4, 5],
       [1, 4, 6],
       [2, 4, 7]]), [(0, 1, 4), (4, 3, 0), (1, 2, 5), (5, 4, 1), (3, 4, 7), (7, 6, 3), (4, 5, 8), (8, 7, 4), (6, 7, 10), (10, 9, 6), (7, 8, 11), (11, 10, 7), (9, 10, 13), (13, 12, 9), (10, 11, 14), (14, 13, 10)])
(array([[0, 0, 1],
       [1, 0, 2],
       [2, 0, 3],
       [0, 1, 2],
       [1, 1, 3],
       [2, 1, 4],
       [0, 2, 3],
       [1, 2, 4],
       [2, 2, 5],
       [0, 3, 4],
       [1, 3, 5],
       [2, 3, 6],
       [0, 4, 5],
       [1, 4, 6],
       [2, 4, 7]]), [(3, 0, 1), (1, 4, 3), (4, 1, 2), (2, 5, 4), (6, 3, 4), (4, 7, 6), (7, 4, 5), (5, 8, 7), (9, 6, 7), (7, 10, 9), (10, 7, 8), (8, 11, 10), (12, 9, 10), (10, 13, 12), (13, 10, 11), (11, 14, 13)])
~~~~

In [53]:
def make_trimesh_feature(z_grid):
    """
    Input: 2D numpy array z_grid of elevation values
    
    Output: Tuple consisting of 2D numpy array of 3D vertex positions
    and a list of triangles represented as tuples of vertex indices 
    for raveled TRIMMED grid
    """
    tri = []
    verts = []
    
    trimmed_grid = z_grid[1:-1, 1:-1]
    
    positions, quads = make_quads(trimmed_grid.shape)
    
    # verts 
    idx = 0 
    for idx_i in range(trimmed_grid.shape[0]):
        for idx_j in range(trimmed_grid.shape[1]):
            verts.append([positions[idx][0], positions[idx][1], trimmed_grid[idx_i][idx_j]])
            idx += 1
    
    #triangles
    computed_grid = compute_features(z_grid)
    grid_shape = trimmed_grid.shape

    idx = 0
    for idx_i in range(grid_shape[0] - 1):
        for idx_j in range(grid_shape[1] - 1):
            diagonal_1 = computed_grid[idx_i][idx_j] + computed_grid[idx_i + 1][idx_j + 1]
            diagonal_2 = computed_grid[idx_i][idx_j + 1] + computed_grid[idx_i + 1][idx_j]

            if diagonal_1 > diagonal_2:
                tri.append((quads[idx][0], quads[idx][1], quads[idx][2]))
                tri.append((quads[idx][2], quads[idx][3], quads[idx][0]))
            else:
                tri.append((quads[idx][3], quads[idx][0], quads[idx][1]))
                tri.append((quads[idx][1], quads[idx][2], quads[idx][3]))
            idx += 1
   
    return (np.array(verts), np.array(tri))

In [11]:
def test_make_trimesh_feature():
    """ Test make_trimesh_feature() """

    make_trimesh_feature(DEMS["ridge"])
    make_trimesh_feature(DEMS["error"])  
    make_trimesh_feature(DEMS["region"])
    
test_make_trimesh_feature()

In [90]:
##################################################################################
# Part 3 - Plot triangular meshes computed from elevation maps


def plot_mesh3d(verts, tris, title="3D plot of a triangular mesh", camera=None):
    """
    Input: 2D numpy array verts of 3D vertex positions, list tris of tuples of vertex indices,
    optional string title, optional dictionary camera
    
    Output: plotly figure corresponing to a triangular mesh created via Mesh3D() 
    using the colorscale ELEV and the specified camera position. 
    The aspectio of the 3D plots should be similar that use in plot_elevation().
    """
    np_tris = np.array(tris)
    
    x_axis = len(verts[:, 0])
    y_axis = len(verts[:, 1])
    
    #compare x and y 
    min_axis = min(x_axis, y_axis)
    x_ratio = x_axis / min_axis
    y_ratio = y_axis / min_axis
    
    data = [go.Mesh3d(x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
                      i=np_tris[:, 0], j=np_tris[:, 1], k=np_tris[:, 2],
                      intensity=verts[:, 2], colorscale=ELEV)]
    
    fig = go.Figure(data=data)
    fig.update_layout(width=800, height=800, title=title,
                      scene={"aspectratio": {"x": x_ratio, "y": y_ratio, "z": 0.1}},
                      scene_camera=camera)
                     
    return fig

In [91]:
def test_plot_mesh3d():
    """ Test plot_mesh3d() """
    
    # Feature-based plots of test function
    verts, tris = make_trimesh_fixed(DEMS["ridge"], diagonal="ul_lr")
    title = "ul_lr triangulation of ridge function (Mesh3D)"
    fig = plot_mesh3d(verts, tris, title)
    fig.show()
    
    verts, tris = make_trimesh_fixed(DEMS["ridge"], diagonal="ll_ur")
    title = "ll_ur triangulation of ridge function (Mesh3D)"
    fig = plot_mesh3d(verts, tris, title)
    fig.show()
    
    verts, tris = make_trimesh_feature(DEMS["ridge"])
    title = "Feature-based triangulation of ridge function (Mesh3D)"
    fig = plot_mesh3d(verts, tris, title)
    fig.show()
    
test_plot_mesh3d()

In [94]:
def plot_trisurf(verts, tris, title="3D plot of a triangular mesh", camera=None):
    """
    Input: 2D numpy array verts of 3D vertex positions, list tris of tuples of vertex indices,
    optional string title, optional dictionary camera
    
    Output: plotly figure corresponding to a triangular mesh create via create_trisurf() 
    using the colormap ELEV and the specified camera position. 
    The aspectio of the 3D plots should be similar that use in plot_elevation().
    """
    
    fig = ff.create_trisurf(x=verts[:, 0], y=verts[:, 1],
                            z=verts[:, 2], simplices=tris, colormap=ELEV)
    
    x_axis = len(verts[:, 0])
    y_axis = len(verts[:, 1])
    
    #compare x and y 
    min_axis = min(x_axis, y_axis)
    x_ratio = x_axis / min_axis
    y_ratio = y_axis / min_axis
    
    fig.update_layout(width=800, height=800, title=title,
                      scene={"aspectratio": {"x": x_ratio, "y": y_ratio, "z": 0.1}},
                      scene_camera=camera)

    return fig

In [95]:
def test_plot_trisurf():
    """ Test plot_trimesh() """
    
    # Feature-based plots of test function
    verts, tris = make_trimesh_fixed(DEMS["ridge"], diagonal="ul_lr")
    title = "ul_lr triangulation of ridge function (trisurf)"
    fig = plot_trisurf(verts, tris, title)
    fig.show()
    
    verts, tris = make_trimesh_fixed(DEMS["ridge"], diagonal="ll_ur")
    title = "ll_ur triangulation of ridge function (trisurf)"
    fig = plot_trisurf(verts, tris, title)
    fig.show()
    
    verts, tris = make_trimesh_feature(DEMS["ridge"])
    title = "Feature-based triangulation of ridge function (trisurf)"
    fig = plot_trisurf(verts, tris, title)
    fig.show()
    
test_plot_trisurf()

## Answer peer-graded question 1 (4 points)

**Examine the plots of the three triangulations for ridge function generated in `test_plot_trisurf()`. Does the feature-based triangulation method do a better job of reproducing the function associated with the original data? (2 pts)? Is one of the fixed triangulations better than the other (2 pts)?**

Yes, the feature-based triangulation method did a better job. The graph is more complete and smooth by choosing the best diagonal for each quad.  

I think the it has no big difference between the two fixed triangulations graphs. The data is simple and the graph are basically the same expect the direction of ridges. 


In [96]:
def plot_canyon_meshes():
    """ Plot a close up of the Grand Canyon DEM on a specific ridge """
    
    # Comparision of region in Grand Canyon with specified camera position
    camera = dict(up=dict(x=0, y=0, z=1),
                  center=dict(x=0, y=0, z=0),
                  eye=dict(x=0.5, y=-0.4, z=0.1))

    verts, tris = make_trimesh_fixed(DEMS["region"], diagonal="ul_lr")
    title = "Close up of ul_lr triangulation of ridge region (trisurf)"
    fig = plot_trisurf(verts, tris, title, camera=camera)
    fig.write_html(PLOTS_PATH + title + ".html")

    verts, tris = make_trimesh_fixed(DEMS["region"], diagonal="ll_ur")
    title = "Close up of ll_ur triangulation of ridge region (trisurf)"
    fig = plot_trisurf(verts, tris, title, camera=camera)
    fig.write_html(PLOTS_PATH + title + ".html")
  
    verts, tris = make_trimesh_feature(DEMS["region"])
    title = "Close-up of feature-based triangulation of ridge region (trisurf)"
    fig = plot_trisurf(verts, tris, title, camera=camera)
    fig.write_html(PLOTS_PATH + title + ".html")
    
    title = "Global view of feature-based triangulation of region (Mesh3D)"
    fig = plot_mesh3d(verts, tris, title)
    fig.write_html(PLOTS_PATH + title + ".html")
    
plot_canyon_meshes()

## Links to the 4 triangular meshes produced by `plot_canyon_meshes()`

* [Close up of ul_lr triangulation of ridge region (trisurf)](plots/Close%20up%20of%20ul_lr%20triangulation%20of%20ridge%20region%20%28trisurf%29.html)
* [Close up of ll_ur triangulation of ridge region (trisurf)](plots/Close%20up%20of%20ll_ur%20triangulation%20of%20ridge%20region%20%28trisurf%29.html)
* [Close-up of feature-based triangulation of ridge region (trisurf)](plots/Close-up%20of%20feature-based%20triangulation%20of%20ridge%20region%20%28trisurf%29.html)
* [Global view of feature-based triangulation of region (Mesh3D)](plots/Global%20view%20of%20feature-based%20triangulation%20of%20region%20%28Mesh3D%29.html)

## Answer peer-graded question 2 (4 points)

**Examine the closeups of the first three triangulations computed in `plot_canyon_meshes()`. Which triangulation does the worst job of reproducing the ridge shown at this camera position (2 pts)? Between the two remaining methods, which method is preferrable? (2 pts)**

At this camera position, the ll lr triangulation is the worst. The ridge is the most obvious one from all the graphs.
The feature-based triangulation is preferrable, the graph is smoother than another graph.


In [17]:
# #Throws an error in create_trisurf()
# print(DEMS["error"])
# verts, tris = make_trimesh_fixed(DEMS["error"], diagonal="ul_lr")
# plot_trisurf(verts, tris, title="Fixed triangulation of error examples")